## Import packages

In [ ]:
import numpy as np
import pandas as pd

## Data Loading

In [ ]:
data_df = pd.read_csv('data.csv')

## Task 1

В першій задачі  зробити "вручну" класифікацію,  використовуючи ядерні оцінки щільності

a) нормальна щільність,  багатовимірний випадок

б) взяти radius.mean  і використати  ядро $K(u)= (\pi)^{-1} sin^2 (u)/u^2$

## Task 2

Записати алгоритм для методу Fix & Hodges.

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.spatial import distance_matrix

In [ ]:
class KNearestNeighbor(BaseEstimator, ClassifierMixin):
    """A kNN classifier"""

    def __init__(self, n_neighbors: int = 1, p: int = 2):
        """
        Args:
            n_neighbors (int): number of neighbors. Defaults to 1.
            p (int): Minkowski p-norm. Defaults to 2.
        """
        self.n_neighbors = n_neighbors
        self.p = p

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        return self._predict_labels(distance_matrix(X, self.X_train, p=self.p))

    def _predict_labels(self, dists):
        """
        Given a matrix of distances between test points and training points,
        predict a label for each test point.
        """
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):

            # Use the distance matrix to find the k nearest neighbors of the ith
            # testing point, and use self.y_train to find the labels of these neighbors
            y_ind = np.argsort(dists[i, :], axis=0)
            closest_y = self.y_train[y_ind[:self.n_neighbors]]

            # Find the most common label in the list closest_y of labels.
            y_pred[i] = np.argmax(np.bincount(closest_y))

        return y_pred